# Preparing Document for Data generation
- This notebook will show you how to do document parsing
- Document Chunking
- And finally mixing it with user QNA to  create seed examples

## Install SDG

```bash 
pip install sdg-hub==0.1.0a2
pip install rich datasets tabulate transformers
```

In [ ]:
%%capture
%pip install rich datasets tabulate transformers
%pip install docling

In [ ]:
import datetime
import os

now = datetime.datetime.now()
timestamp = now.strftime('%Y%m%d-%H%M%S')

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

## Select a document

### 2024 IBM Annual Report

In [ ]:
data_dir = 'document_collection/ibm-annual-report'
user_config_path = f'{data_dir}/qna.yaml'
list_md_files = [f"{data_dir}/ibm-annual-report-2024.md"]

### Teigaku Genzei

In [ ]:
data_dir = 'document_collection/teigaku-genzei'
user_config_path = f'{data_dir}/qna_ja.yaml'
list_md_files = [f"{data_dir}/0024001-021.md"]

## Initialize common variables

In [ ]:
data_name = os.path.basename(data_dir)

seed_data_path = f"{data_dir}/seed_data_{data_name}.jsonl"
# seed_data_path = f"{data_dir}/seed_data_{data_name}_{timestamp}.jsonl"

## Pre-process a document using docling v2

In [ ]:
# !OMP_NUM_THREADS=32 mamba run -n docling python ../scripts/docparser_v2.py --input-dir {data_dir} --output-dir {data_dir} --c docling_v2_config.yaml
!python docparser_v2.py --input-dir {data_dir} --output-dir {data_dir} -c docling_v2_config.yaml

## Create Seed Examples

In [ ]:
from sdg_hub.utils.docprocessor import DocProcessor

dp = DocProcessor(data_dir, user_config_path=user_config_path)

### Using markdown file
# Note: For now v2 json is not supported
seed_data = dp.get_processed_markdown_dataset(list_md_files)

# seed_data.to_json(seed_data_path, orient='records', lines=True)
seed_data.to_json(seed_data_path, orient='records', lines=True, force_ascii=False)